In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

#other imports
import string
import re
import numpy as np
import pandas as pd

In [2]:
df2 = pd.read_excel("02_pagina12.xlsx")
df2['Diario'] = 'Pagina12'
df3 = pd.read_excel("03_Infobae.xlsx")
df3['Diario'] = 'Infobae'

In [3]:
df = pd.concat([df2,df3])

In [20]:
# si ya estan instalados los modelos para spacy/nltk, ignorar
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('vader_lexicon')

In [10]:
doc_complete = df['contenido']

In [11]:
stop = set(stopwords.words('spanish'))

In [12]:
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete]  

In [13]:
dictionary = corpora.Dictionary(doc_clean)

In [14]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [15]:
Lda = gensim.models.ldamodel.LdaModel
#Lda = gensim.models.ldamulticore.LdaModel


In [16]:
ldamodel = Lda(doc_term_matrix, num_topics=16, id2word = dictionary, passes=50)

In [17]:
df_lda = pd.DataFrame(ldamodel.print_topics(num_topics=16, num_words=5))

In [18]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [19]:
sid = SentimentIntensityAnalyzer()

In [87]:
for i in range(0,1000):
    text = str(df.at[i, 'contenido'])
    scores = sid.polarity_scores(text)
    print(scores)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.893, 'pos': 0.107, 'compound': 0.34}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.095, 'neu': 0.905, 'pos': 0.0, 'compound': -0.296}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.897, 'pos': 0.103, 'compound': 0.34}
{'neg': 0.104, 'neu': 0.896, 'pos': 0.0, 'compound': -0.296}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.104, 'neu': 0.896, 'pos': 0.0, 'compound': -0.296}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.

In [ ]:
import spacy
from transformers import pipeline

In [95]:
# Load the SpaCy English model
nlp = spacy.load("es_core_news_lg") #es_core_news_lg modelo large, el transformer es es_dep_news_trf
# Define a text for sentiment analysis
text = "neutro"
# Create a sentiment analysis pipeline using a pre-trained model
sentiment_analysis = pipeline("sentiment-analysis")
# Perform sentiment analysis on the text
result = sentiment_analysis(text)
# Print the sentiment result
print("Sentiment:", result[0]['label'], result[0]['score'])


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Sentiment: NEGATIVE 0.9847928285598755


In [97]:
for i in range(0,20):
    text = str(df.at[i, 'copete'])
    scores = sentiment_analysis(text)[0]
    print(f"Indice: {i}\nTexto: {text:10} \nSentimiento: {scores['label']} Puntaje: {scores['score']}\n")

Indice: 0
Texto: 0    Buenos Aires12Diego Hurtado, experto en energí...
0    Buenos Aires, 29 ene (EFE).- El bloque del fre...
Name: copete, dtype: object 
Sentimiento: NEGATIVE Puntaje: 0.7760317921638489

Indice: 1
Texto: 1    El paísEl ex ministro de Cultura desmiente una...
1    Desde Roma, donde tiene previsto participar de...
Name: copete, dtype: object 
Sentimiento: NEGATIVE Puntaje: 0.9676669239997864

Indice: 2
Texto: 2    El paísEncuesta especial del CEOP para Página/...
2    En una extensa entrevista que Javier Milei bri...
Name: copete, dtype: object 
Sentimiento: NEGATIVE Puntaje: 0.973739504814148

Indice: 3
Texto: 3    El país  Javier MileiEl Presidente en su salsa...
3    Washington, 16 feb (EFE).- El secretario de Es...
Name: copete, dtype: object 
Sentimiento: NEGATIVE Puntaje: 0.7102692723274231

Indice: 4
Texto: 4    Buenos Aires12Brenda Vargas Matyi, diputada na...
4    Durante la tarde del sábado, el presidente Jav...
Name: copete, dtype: object 
Sentimiento: NEGA

In [139]:
from transformers import pipeline

# Load a sentiment analysis pipeline
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

# 1 star: Very negative sentiment.
# 2 stars: Negative sentiment.
# 3 stars: Neutral sentiment.
# 4 stars: Positive sentiment.
# 5 stars: Very positive sentiment.

In [152]:
# Example Spanish text
text = ""

# Perform sentiment analysis
results = classifier(text)

# Print the results
print(results)

[{'label': '1 star', 'score': 0.512035071849823}]


In [118]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
# Create a pipeline for sentiment analysis
# Note: This step assumes that you have a model fine-tuned for sentiment analysis. 
# If you're using BETO directly without fine-tuning, replace 'sentiment-analysis' with another suitable task, 
# like 'feature-extraction' or prepare to fine-tune the model on a sentiment analysis dataset.
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [128]:
# Example text
text = "Muy bueno"

# Perform sentiment analysis
results = sentiment_pipeline(text)

# Display the results
print(results)

[{'label': 'LABEL_0', 'score': 0.5633345246315002}]
